# Max Flow implementation
### Raw data

In [1]:
data=readcsv("data/maxflowdata.csv")

8×3 Array{Any,2}:
  "s"  1      5
  "s"  2      4
 1      " t"  8
 1     3      6
 2     1      3
 2     3      2
 2      " t"  4
 3      " t"  1

Now we need to do a bit of data cleaning. We will:
1. Replace "s" by 1
2. Increment all node numbers
3. Replace "t" by maximum node number+1

In [2]:
# Steps 1 and 2..usafe
for i=1:size(data,1), j=1:2
    if data[i,j]=="s"
        data[i,j]=1
    elseif typeof(data[i,j])<:Number
        data[i,j]+=1
    end 
end

In [3]:
# Safe way
n_data=copy(data)
max_node_num=0
for i=1:size(data,1), j=1:2
    if data[i,j]=="s"
        n_data[i,j]=1
    elseif typeof(data[i,j])<:Number
        n_data[i,j]+=1
        (max_node_num<n_data[i,j]) && (max_node_num=n_data[i,j])
    end 
end
max_node_num+=1
# Now we do step three
for i=1:size(data,1), j=1:2
    if data[i,j]==" t"
        n_data[i,j]=max_node_num
    end
end
n_data

8×3 Array{Any,2}:
 2  3  5
 2  4  4
 3  6  8
 3  5  6
 4  3  3
 4  5  2
 4  6  4
 5  6  1

Deepcopy vs copy

In [4]:
a=[[1,2],[3,4]]

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]

In [5]:
b=copy(a)

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]

In [6]:
b[1][2]=4

4

In [7]:
b

2-element Array{Array{Int64,1},1}:
 [1, 4]
 [3, 4]

In [8]:
a

2-element Array{Array{Int64,1},1}:
 [1, 4]
 [3, 4]

In [9]:
a=[[1,2],[3,4]]
b=deepcopy(a)
b[1][2]=4
a

2-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]

In [10]:
b

2-element Array{Array{Int64,1},1}:
 [1, 4]
 [3, 4]

Creating an arc structure

In [11]:
type Arc
  from::Int
  to::Int
  capacity::Int
  flow::Int
end

In [12]:
# lo=Array{Arc,1}[]
# for i=1:max_node_num push!(lo,Arc[]) end
AdjL=Array{Array{Arc,1}}(max_node_num)
for i=1:max_node_num AdjL[i]=Arc[] end
for i=1:size(n_data,1)
    from, to, cap =n_data[i,1], n_data[i,2],n_data[i,3]
    push!(AdjL[from],Arc(from,to,cap,0))
end
AdjL   

6-element Array{Array{Arc,1},1}:
 Arc[]                                                 
 Arc[Arc(2, 3, 5, 0), Arc(2, 4, 4, 0)]                 
 Arc[Arc(3, 6, 8, 0), Arc(3, 5, 6, 0)]                 
 Arc[Arc(4, 3, 3, 0), Arc(4, 5, 2, 0), Arc(4, 6, 4, 0)]
 Arc[Arc(5, 6, 1, 0)]                                  
 Arc[]                                                 

In [13]:
function createResNet(AdjL)
    resAdjL=deepcopy(AdjL)
    for i in eachindex(AdjL)
        for arc in AdjL[i]
            push!(resAdjL[arc.to],Arc(arc.to,arc.from,0,0))
        end
    end
    resAdjL
end

createResNet (generic function with 1 method)

In [14]:
resAdjL=createResNet(AdjL)

6-element Array{Array{Arc,1},1}:
 Arc[]                                                                  
 Arc[Arc(2, 3, 5, 0), Arc(2, 4, 4, 0)]                                  
 Arc[Arc(3, 6, 8, 0), Arc(3, 5, 6, 0), Arc(3, 2, 0, 0), Arc(3, 4, 0, 0)]
 Arc[Arc(4, 3, 3, 0), Arc(4, 5, 2, 0), Arc(4, 6, 4, 0), Arc(4, 2, 0, 0)]
 Arc[Arc(5, 6, 1, 0), Arc(5, 3, 0, 0), Arc(5, 4, 0, 0)]                 
 Arc[Arc(6, 3, 0, 0), Arc(6, 4, 0, 0), Arc(6, 5, 0, 0)]                 

In [15]:
function depthFirstSearch(start_node,end_node,AdjL)
    # Initialization
    n=length(AdjL) # O(1)
    pred=zeros(Int,n) # O(n)
    order=zeros(Int,n)  # O(n)
    marked=zeros(Bool,n)  # O(n)
    next=1  # O(1)
    order[start_node]=next  # O(1)
    marked[start_node]=true  # O(1)
    LIST=[start_node]  # O(1)... 
    # Main search part
    while !isempty(LIST)
        i=LIST[end]
        found,j=find_admissable(i,marked,AdjL)
        if found
            marked[j]=true
            pred[j]=i
            next+=1
            order[j]=next
            push!(LIST,j) # O(1) most of the time...possibly optimization by having the LIST of size n
            j==end_node && break
        else
            pop!(LIST) # Would be pop! for depth first
        end
    end
    # Returning 
    currentNode=end_node
    path=[currentNode]
    while currentNode!=start_node
        if (currentNode==0)
            path=Int[]
            break
        end
        currentNode=pred[currentNode]
        unshift!(path,currentNode)
    end
    path 
end

depthFirstSearch (generic function with 1 method)

In [16]:
function find_admissable(i,marked,AdjL)
    for arc in AdjL[i]
        if arc.capacity>0 && !marked[arc.to] 
            return true, arc.to
        end
    end
    false,0
end

find_admissable (generic function with 1 method)

In [17]:
depthFirstSearch(5,1,resAdjL)

0-element Array{Int64,1}